In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Basics").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
url ='https://hdietrichawsbucket.s3.ca-central-1.amazonaws.com/employee.csv'

In [ ]:
from pyspark import SparkFiles

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("employee.csv"), sep=",", header=True)


In [ ]:
df.show()

+---+-----------+--------+--------------------+--------+--------------+------+---------+--------+--------------+----------------+--------------------+------------+
|_c0|Employee ID|   Email|            Position|     DOB|Marital Status|Gender|Hire Date|Salaried|Vacation Hours|Sick Leave Hours|  Encrypted Password|    Modified|
+---+-----------+--------+--------------------+--------+--------------+------+---------+--------+--------------+----------------+--------------------+------------+
|  1|  295847284|    ken0|Chief Executive O...| 1/29/69|             S|     M|  1/14/09|       1|            99|              69|F01251E5-96A3-448...|6/30/14 0:00|
|  2|  245797967|  terri0|Vice President of...|  8/1/71|             S|     F|  1/31/08|       1|             1|              20|45E8F437-670D-440...|6/30/14 0:00|
|  3|  509647174|roberto0| Engineering Manager|11/12/74|             M|     M| 11/11/07|       1|             2|              21|9BBBFB2C-EFBB-421...|6/30/14 0:00|
|  4|  112457891

In [ ]:
server= "bootcampdb.cmwhqarpyd9v.ca-central-1.rds.amazonaws.com"
database = "postgres"
port ="5432"
username = "root"
password = "xy92Hdie"

In [ ]:
conn = f"postgres://{username}:{password}@{server}:{port}/{database}"

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(conn, echo=False)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
pandas_df = df.toPandas()

In [ ]:
pandas_df.head()

,_c0,Employee ID,Email,Position,DOB,Marital Status,Gender,Hire Date,Salaried,Vacation Hours,Sick Leave Hours,Encrypted Password,Modified
0,1,295847284,ken0,Chief Executive Officer,1/29/69,S,M,1/14/09,1,99,69,F01251E5-96A3-448D-981E-0F99D789110D,6/30/14 0:00
1,2,245797967,terri0,Vice President of Engineering,8/1/71,S,F,1/31/08,1,1,20,45E8F437-670D-4409-93CB-F9424A40D6EE,6/30/14 0:00
2,3,509647174,roberto0,Engineering Manager,11/12/74,M,M,11/11/07,1,2,21,9BBBFB2C-EFBB-4217-9AB7-F97689328841,6/30/14 0:00
3,4,112457891,rob0,Senior Tool Designer,12/23/74,S,M,12/5/07,0,48,80,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,6/30/14 0:00
4,5,695256908,gail0,Design Engineer,9/27/52,M,F,1/6/08,1,5,22,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,6/30/14 0:00


In [ ]:
pandas_df.to_sql("employee", con=engine, if_exists = 'replace')

In [ ]:
import pandas as pd
final_df = pd.read_sql_table("employee", conn)
final_df.head()

,index,_c0,Employee ID,Email,Position,DOB,Marital Status,Gender,Hire Date,Salaried,Vacation Hours,Sick Leave Hours,Encrypted Password,Modified
0,0,1,295847284,ken0,Chief Executive Officer,1/29/69,S,M,1/14/09,1,99,69,F01251E5-96A3-448D-981E-0F99D789110D,6/30/14 0:00
1,1,2,245797967,terri0,Vice President of Engineering,8/1/71,S,F,1/31/08,1,1,20,45E8F437-670D-4409-93CB-F9424A40D6EE,6/30/14 0:00
2,2,3,509647174,roberto0,Engineering Manager,11/12/74,M,M,11/11/07,1,2,21,9BBBFB2C-EFBB-4217-9AB7-F97689328841,6/30/14 0:00
3,3,4,112457891,rob0,Senior Tool Designer,12/23/74,S,M,12/5/07,0,48,80,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,6/30/14 0:00
4,4,5,695256908,gail0,Design Engineer,9/27/52,M,F,1/6/08,1,5,22,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,6/30/14 0:00


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-03-12 01:43:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.5’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-03-12 01:43:24 (5.99 MB/s) - ‘postgresql-42.2.9.jar.5’ saved [914037/914037]



In [ ]:
mode = 'append'
jdbc_conn = f"jdbc:postgresql://{server}:{port}/{database}"
config = {"user":"root", "password":"xy92Hdie","driver":"org.postgresql.Driver"}

In [ ]:
df.write.jdbc(url=jdbc_conn, table="employee1", mode=mode, properties=config)